<a href="https://colab.research.google.com/github/gtbnhyujmj/-Good-Auto_Shorts_Maker/blob/main/%5BV2%5D_Stickered_Movie_%E8%A8%BB%E8%A7%A3%E5%8A%A0%E6%95%99%E5%AD%B8%E7%89%88_%E5%88%9D%E7%A8%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 影像疊貼自動產生短影音（新手逐行解釋版）
本 Notebook 會教你怎麼自動把 PNG 貼紙貼到背景影片上，產生直式短影音。每一格都會有很詳細的新手註解！


## 1. 掛載 Google Drive

**這一段是做什麼？**
- 在 Google Colab 上，預設只能存取 `/content` 目錄，這個目錄用完就會消失。
- 如果你要讀寫你自己的檔案（例如背景圖、素材、輸出影片），建議全部都放在 Google 雲端硬碟，這樣 Colab 才能存/讀/寫。

**怎麼用？**
- 執行這一格，Colab 會請你授權存取你的 Google Drive。授權一次後，`/content/drive/MyDrive` 就會是你雲端硬碟的主目錄。

**新手常見錯誤**
- 每次重開 Colab 都要重新掛載。
- 如果沒有掛載就直接用硬碟路徑，程式會找不到資料夾（會 error）。


In [ ]:
from google.colab import drive  # 匯入 Colab 的 Google Drive 模組
drive.mount('/content/drive')  # 掛載 Google 雲端硬碟到 /content/drive，這樣才能讀寫雲端檔案



## 2. 匯入必要的 Python 套件

**這一段是做什麼？**
- 本專案會處理影片（背景）、圖片（人物/貼紙），還有資料夾與檔案操作。
- 各套件用途如下：

| 套件         | 用途說明                                  | 常見錯誤              |
|--------------|----------------------------------------|--------------------|
| `os`         | 跟作業系統相關，找路徑/檔名/資料夾管理           | 寫路徑打錯、路徑斜線方向 |
| `cv2`        | OpenCV，專門做圖片和影片處理                   | Colab 初次用要 pip 安裝，版本問題 |
| `numpy`      | 做矩陣、影像陣列運算（很多影像套件都吃 numpy 格式） | 陣列資料型態錯誤        |
| `PIL.Image`  | Python 內建的圖片處理（Pillow）                | 讀入路徑打錯、格式不合   |
| `random`     | 隨機抽檔案、打亂順序等用途                      | -                  |

**注意：在 Colab 若出現「ModuleNotFoundError」就 pip install 一下，例如：`!pip install opencv-python`**


In [ ]:
import os  # Python 內建作業系統功能：處理檔案、資料夾、路徑
import cv2  # OpenCV：強大的圖片/影片讀寫、轉檔、處理（初學者常見於影像辨識）
import numpy as np  # Numpy：處理大量數據、圖片像素運算都會用到
from PIL import Image  # Pillow：進階圖片處理、支援 PNG 透明度
import random  # 亂數模組：抽背景、角色、貼紙等素材時會用

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os  # 匯入標準/外部函式庫
import cv2  # 匯入標準/外部函式庫
import numpy as np  # 匯入標準/外部函式庫
from PIL import Image  # 從套件匯入模組或類別
import random  # 匯入標準/外部函式庫

In [ ]:
# ===== 函數：合成透明圖層（快速 numpy） =====
def blend_with_alpha_fast(base_img, overlay_imgs):  # 定義函數：blend_with_alpha_fast
    base_rgba = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGBA)  # OpenCV 影像處理函式
    for overlay in overlay_imgs:  # for 迴圈
        overlay_rgba = np.array(overlay.convert('RGBA').resize((base_rgba.shape[1], base_rgba.shape[0]), Image.LANCZOS))  # numpy 相關函式
        alpha_overlay = overlay_rgba[..., 3:] / 255.0  # 變數指派
        alpha_base = base_rgba[..., 3:] / 255.0  # 變數指派
        out_alpha = alpha_overlay + alpha_base * (1 - alpha_overlay)  # 變數指派
        base_rgba[..., :3] = (overlay_rgba[..., :3] * alpha_overlay + base_rgba[..., :3] * alpha_base * (1 - alpha_overlay)) / (out_alpha + 1e-6)  # 變數指派
        base_rgba[..., 3:] = out_alpha * 255  # 變數指派
    result_bgr = cv2.cvtColor(base_rgba, cv2.COLOR_RGBA2BGR)  # OpenCV 影像處理函式
    return result_bgr  # 回傳結果

In [ ]:
# ===== 函數：處理影片，每幀合成 PNG 疊圖 =====
def overlay_multiple_pngs_on_video_fast(video_path, overlay_paths, output_path):  # 定義函數：overlay_multiple_pngs_on_video_fast
    cap = cv2.VideoCapture(video_path)  # OpenCV 影像處理函式
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # OpenCV 影像處理函式
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # OpenCV 影像處理函式
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # OpenCV 影像處理函式
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # OpenCV 影像處理函式
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))  # OpenCV 影像處理函式
    overlay_images = [Image.open(path).resize((width, height), Image.LANCZOS) for path in overlay_paths]  # PIL 影像處理
    while True:
        ret, frame = cap.read()  # 變數指派
        if not ret:  # if 判斷式
            break
        result_frame = blend_with_alpha_fast(frame, overlay_images)  # 變數指派
        out.write(result_frame)
    cap.release()
    out.release()
    print(f"✅ 影片已產出：{output_path}")


## 3. 定義圖層合成函數：`blend_with_alpha_fast`

**這一段是做什麼？**
- 負責把底圖跟多張 PNG 疊起來，而且會考慮透明度（alpha channel），所以效果像貼紙。
- 用 numpy 讓速度更快，適合一次合成很多圖層。

**新手要注意什麼？**
- `base_img` 要是 OpenCV 的格式（BGR），不是 PIL 格式。
- `overlay_imgs` 要是 PIL 圖片，而且本來就要有透明度（RGBA）。
- PIL 跟 OpenCV 兩套影像格式常常轉來轉去，一定要清楚目前是哪一種（錯就會出現 shape/型態錯誤）。


In [ ]:
# ===== 資料夾設定 =====
BACKGROUND_DIR = "/content/drive/MyDrive/shorts/backgrounds_MP4"  # 可包含 MP4 或 PNG  # 變數指派
CHARACTER_DIR = "/content/drive/MyDrive/shorts/characters"  # 變數指派
OVERLAY_DIR = "/content/drive/MyDrive/shorts/PNG_Overlayers"  # 變數指派
TEXT_STICKER_DIR = "/content/drive/MyDrive/shorts/Text_Overlayers"  # 變數指派
OUTPUT_DIR = "/content/drive/MyDrive/shorts/movie01"  # 變數指派
os.makedirs(OUTPUT_DIR, exist_ok=True)  # 操作路徑或檔案系統


## 4. 定義「疊 PNG 到影片」主函數：`overlay_multiple_pngs_on_video_fast`

**這一段是做什麼？**
- 讀進一段 MP4 影片，針對影片的每一幀（frame），把多個 PNG 疊貼上去（就像字幕、貼紙）。
- 疊完後，把所有新 frame 再輸出成新影片。

**新手注意**：
- 這段是「每一幀」處理，影片長越久，越慢。
- 注意記憶體（幀數大會吃很大）。
- 輸入/輸出路徑要小心，否則找不到/覆蓋舊檔。


In [ ]:
# ===== 主程式流程：對每張貼紙產出影片 =====
text_files = [f for f in os.listdir(TEXT_STICKER_DIR) if f.endswith('.png')]  # 操作路徑或檔案系統
if not text_files:  # if 判斷式
    print("⚠️ 找不到任何貼紙，請確認 TEXT_STICKER_DIR 資料夾內是否有 PNG。")

for text_file in text_files:  # for 迴圈
    # 讀入文字貼紙
    text_path = os.path.join(TEXT_STICKER_DIR, text_file)  # 操作路徑或檔案系統

    # 隨機抽背景
    bg_list = [os.path.join(BACKGROUND_DIR, f) for f in os.listdir(BACKGROUND_DIR) if f.endswith(('.mp4', '.png'))]  # 操作路徑或檔案系統
    if not bg_list:  # if 判斷式
        print("❌ 沒有背景素材，請確認背景資料夾。")
        continue
    bg_path = random.choice(bg_list)  # 隨機數/物件

    # 隨機抽角色與特效
    def rand_img(folder):  # 定義函數：rand_img
        items = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.png')]  # 操作路徑或檔案系統
        return random.choice(items) if items else None  # 回傳結果
    layers = [text_path]  # 最底層為文字貼紙  # 變數指派
    for folder in [CHARACTER_DIR, OVERLAY_DIR]:  # for 迴圈
        picked = rand_img(folder)  # 變數指派
        if picked:  # if 判斷式
            layers.append(picked)

    # 合成影片
    output_path = os.path.join(OUTPUT_DIR, text_file.replace(".png", ".mp4"))  # 操作路徑或檔案系統
    if bg_path.endswith('.mp4'):  # if 判斷式
        overlay_multiple_pngs_on_video_fast(bg_path, layers, output_path)
    else:  # else 其他情況
        # 處理靜態背景圖
        bg_img = cv2.imread(bg_path)  # OpenCV 影像處理函式
        bg_img = cv2.resize(bg_img, (1080, 1920))  # OpenCV 影像處理函式
        overlay_imgs = [Image.open(p).resize((1080, 1920), Image.LANCZOS) for p in layers]  # PIL 影像處理
        result = blend_with_alpha_fast(bg_img, overlay_imgs)  # 變數指派
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 15, (1080, 1920))  # OpenCV 影像處理函式
        for _ in range(75): out.write(result)  # for 迴圈
        out.release()
        print(f"✅ 靜態背景影片產出：{output_path}")

print("🎬 所有影片產出完成！")

✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_0.mp4
✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_1.mp4
✅ 影片已產出：/content/drive/MyDrive/shorts/movie01/text_2.mp4
🎬 所有影片產出完成！
